# Backpropagation Neural Networks - Learning and Implementation strategies

In this notebook, we will study the use of `PyTorch` and `Tensorflow` frameworks for implementing and training Neural Networks. This is not intended to be exhaustive, but rather to provide examples for exploring the algorithms and their hyperparameters with these frameworks.


### Imports

In [1]:
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

In [4]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from mlxtend.plotting import plot_decision_regions
from tqdm import tqdm

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

### Global settings

In [6]:
seed = 1
device = 'cpu'

### XOR (exclusive OR) data simulation

Let's run our examples for a very simple non-linear binary classification example.

In [7]:
# Data simulation
N_data = 200
train_size = 150
X = 2 * torch.rand(N_data, 2, device=device, dtype=torch.float32) - 1
y = torch.tensor([0 if elem[0]*elem[1] < 0 else 1 for elem in X], device=device, dtype=torch.float32)

# Split training and test partitions
X_train = X[:train_size, :]
y_train = y[:train_size]
X_val = X[train_size:, :]
y_val = y[train_size:]

# Define datasets for data loaders
train_ds = TensorDataset(X_train, y_train)
val_ds = TensorDataset(X_val, y_val)

# Create the data loaders
batch_size_GD = train_size
train_dl_GD = DataLoader(train_ds, batch_size_GD, shuffle=True)
val_dl_GD = DataLoader(val_ds, batch_size_GD, shuffle=True)

batch_size_SGD = 1
train_dl_SGD = DataLoader(train_ds, batch_size_SGD, shuffle=True)
val_dl_SGD = DataLoader(val_ds, batch_size_SGD, shuffle=True)

batch_size_MiniSGD = 32
train_dl_MniSGD = DataLoader(train_ds, batch_size_MiniSGD, shuffle=True)
val_dl_MiniSGD = DataLoader(val_ds, batch_size_MiniSGD, shuffle=True)

print('Partition sizes')
print([X_train.shape, y_train.shape], [X_train.dtype, y_train.dtype], [X_train.device, y_train.device])
print([X_val.shape, y_val.shape], [X_val.dtype, y_val.dtype], [X_val.device, y_val.device])

print('\nBatch sizes')
print(batch_size_GD)
print(batch_size_SGD)
print(batch_size_MiniSGD)

Partition sizes
[torch.Size([150, 2]), torch.Size([150])] [torch.float32, torch.float32] [device(type='cpu'), device(type='cpu')]
[torch.Size([50, 2]), torch.Size([50])] [torch.float32, torch.float32] [device(type='cpu'), device(type='cpu')]

Batch sizes
150
1
32


In [8]:

X_np = X.cpu().numpy()
y_np = y.cpu().numpy()

fig = go.Figure()

# Class 0
fig.add_trace(go.Scatter(
    x=X_np[y_np == 0, 0],
    y=X_np[y_np == 0, 1],
    mode='markers',
    marker=dict(color='blue', size=9, opacity=0.35),
    name='Class 0'
))

# Class 1
fig.add_trace(go.Scatter(
    x=X_np[y_np == 1, 0],
    y=X_np[y_np == 1, 1],
    mode='markers',
    marker=dict(color='red', size=9, opacity=0.35),
    name='Class 1'
))

fig.add_shape(type='line', x0=0, x1=0, y0=-1.75, y1=1.75, line=dict(color='black', dash='dot'))
fig.add_shape(type='line', x0=-1.75, x1=1.75, y0=0, y1=0, line=dict(color='black', dash='dot'))
fig.update_layout(
    width=550,
    height=400,
    xaxis=dict(range=[-1.25, 1.25], title=r'$x_1$'),
    yaxis=dict(range=[-1.25, 1.25], title=r'$x_2$'),
    legend=dict(x=1, y=1),
    margin=dict(l=20, r=20, t=20, b=20)
)

fig.show()


<br />
<hr />

## 1. Building a Network with PyTorch

Check the <a href='https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module'>PyTorch documentation</a> for details.

To build a Neural Network in PyTorch, we create new Classes that inherit from the `nn.Module` class.

This, some functionalities are already implemented. However, the model definitions have to be made mainly by hand. As a general rule, following the setps:

1. Define the architecture of the network;
2. Initialize the weights and biases of the network;
3. Define the forward pass of the network;
4. Define the training loop behaviour;
5. Create the NN instance;
6. Define the loss function;
7. Define the optimizer.


### 1. 1. Network architecture

Let's create a NN with three **linear layers**:

1. The first will upscale the 2 input layer nodes (2 features in the dataset) to 3 nodes.
2. The second will upscale to 4 nodes.
3. The third (output) will, then, downscale to 1 node.

Besides that, the **activation functions** should be:

1. The ReLU function in the hidden layer, and
2. The sigmoid function in the output layer.

```python
class Net(nn.Module):
    def __init__(self):
        # Instantiate the parent (nn.Module) class
        super(Net, self).__init__()
    
        # NN architecture definition
        ...

```
<br />

### 1.2. Weights and bias initialization

For the weights and bias initialization, let's use methods that are already available in the PyTorch library. However, notice that you can easily implement your own initialization methods.

- Weights: let's use the Xavier Uniform Initialization method. It maintains the variance of the activations remain the same across the layers of the network.
- Bias: bias will be initializes with zeros.

Take a look at the PyTorch docs for more available initialization methods:
<a href='https://pytorch.org/docs/stable/nn.init.html'>https://pytorch.org/docs/stable/nn.init.html</a>.

```python
torch.nn.init.xavier_uniform_(attribute.weight)
torch.nn.init.zeros_(attribute.bias)

```
<br />

### 1. 3. Forward pass

This step sets how the linear combination of the inputs and weights of each layer should work and how the combination of the linear step should be combined with the activation functions:

```python
def forward(self, x):
    # For each layer, the output will be the ReLu activation applied to the output of the linear operation
    x = self.activation(self.fc1(x))
    # For the last layer, the sigmoid function will be the activation
    x = torch.sigmoid(self.fc2(x))
    return x
```
<br />

### 1. 4. Training loop

Having defined the Neural Network topology, the initialization method, and the feedforward pass, the behavior of the backpropagation should be set in the training loop.

For that, Pytorch has some useful methods:

- The `backward()` method calculates the derivative of the Error in respect to the NN weights applying the chain rule for hidden neurons;
- The `step()` method updates the weights and bias based on the computed gradients.

<br />

Let's put steps 1 to 4 altogether:

<br />

# read from here

In [9]:
import time
class Net_group_Y(nn.Module):
    
    def __init__(self, input_size=2, output_size=1,
                 hidden_layer_sizes=[3,4,5], activation=nn.ReLU()):
        
        super(Net_group_Y, self).__init__()
        #
        # 1. 1. Network architecture
        
        self.add_module(f'fc{1}', nn.Linear(input_size, hidden_layer_sizes[0]))
        
        for i in range(1,len(hidden_layer_sizes)):
            self.add_module(f'fc{i+1}', nn.Linear(hidden_layer_sizes[i-1], hidden_layer_sizes[i]))
        
        self.add_module(f'fc{len(hidden_layer_sizes) +1 }', nn.Linear(hidden_layer_sizes[-1], output_size))

        # Weights initialisation
        # The apply method applies the function passed as the apply() argument
        # to each element in the object, that in this case is the neural network.
        self.apply(self._init_weights)
        # Store the parameters
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        
        self.n_forward_calls = 0
        
    #
    # 1. 2. Weights and bias initialization
    #
    def _init_weights(self, attribute):
        if isinstance(attribute, nn.Linear):
          torch.nn.init.xavier_uniform_(attribute.weight)
          torch.nn.init.zeros_(attribute.bias)
    #
    # 1. 3. Forward pass
    """    
    def forward(self, x):
        # For each layer, the output will be the ReLu activation applied to the output of the linear operation
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        # For the last layer, the sigmoid function will be the activation
        x = torch.sigmoid(self.fc3(x))
        return x"""
    
    def forward(self, x):
        # Forward pass through all layers
        for i in range(1, len(self.hidden_layer_sizes) + 2):
            #print(f'forward pass layer {i}')
            layer = getattr(self, f'fc{i}')
            x = layer(x)
            if i < len(self.hidden_layer_sizes):
                self.n_forward_calls += 1
                x = self.activation(x)
        # Apply sigmoid activation to the output layer
        self.n_forward_calls += 1
        x = torch.sigmoid(x)
        return x
    
    #
    # 1. 4. Training loop
    # For details, see Machine Learning with PyTorch and Scikit-Learn.
    #
    def train(self, num_epochs, loss_fn, optimizer, train_dl, train_size, batch_size, x_valid, y_valid):
        # Initialize weights
        self.apply(self._init_weights)
    
        # Loss and accuracy history objects initialization
        loss_hist_train = [0] * num_epochs
        accuracy_hist_train = [0] * num_epochs
        loss_hist_valid = [0] * num_epochs
        accuracy_hist_valid = [0] * num_epochs
        delta_times = [0] * num_epochs
        self.n_forward_calls = 0
        
        # Learning loop
        for epoch in tqdm(range(num_epochs)):
            start_time = time.time()
            # Batch learn
            for x_batch, y_batch in train_dl:
                #print('*'*20)
                # ---
                # 1.4.1. Get the predictions, the [:,0] reshapes from (batch_size,1) to (batch_size)
                pred = self(x_batch)[:,0]
                # 1.4.2. Compute the loss
                loss = loss_fn(pred, y_batch)
                # 1.4.3. Back propagate the gradients
                # The `backward()` method, already available in PyTroch, calculates the 
                # derivative of the Error in respect to the NN weights
                # applying the chain rule for hidden neurons.
                loss.backward()
                # 1.4.4. Update the weights based on the computed gradients
                optimizer.step()
                # ---
                
                # Reset to zero the gradients so they will not accumulate over the mini-batches
                optimizer.zero_grad()
                
                # Update performance metrics
                loss_hist_train[epoch] += loss.item()
                is_correct = ((pred>=0.5).float() == y_batch).float()
                accuracy_hist_train[epoch] += is_correct.mean()
            
            delta_times[epoch] = time.time() - start_time
            # Average the results
            loss_hist_train[epoch] /= train_size/batch_size
            accuracy_hist_train[epoch] /= train_size/batch_size
            
            # Predict the validation set
            pred = self(x_valid)[:, 0]
            loss_hist_valid[epoch] = loss_fn(pred, y_valid).item()
            is_correct = ((pred>=0.5).float() == y_valid).float()
            accuracy_hist_valid[epoch] += is_correct.mean()
            
        return loss_hist_train, loss_hist_valid, accuracy_hist_train, accuracy_hist_valid, self.n_forward_calls, delta_times

    # Not needed normaly, it is just for mlextend plots
    def predict(self, x):
        print(f'predict with input shape: {x.shape}')
        x = torch.tensor(x, dtype=torch.float32)
        pred = self.forward(x)[:, 0]
        print(f'finished predict with output shape: {pred.shape}')
        return (pred>=0.5).float()
        

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

criterion = RMSELoss()

class nn_model():
    def __init__(self,
                X_train,
                X_val,
                y_train,
                y_val,
                input_size, 
                output_size,
                hidden_layer_sizes, 
                optimizer,
                num_epochs,
                train_dl,  
                train_size,
                batch_size,
                learning_rate,
                activation=nn.ReLU()):
        
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_dl = train_dl
        self.train_size = train_size
        self.batch_size = batch_size
        
        self.arcthicture = Net_group_Y(input_size=input_size,
                                  output_size=output_size,
                                  hidden_layer_sizes=hidden_layer_sizes,
                                  activation=activation)
        
        self.arcthicture.to(device)
        self.optimizer = optimizer
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        
        self.fitness, self.test_fitness, self.accuracy, self.accuracy_valid, self.n_forward_calls, self.delta_times = self.fit(X_train, X_val, y_train, y_val, arcthicture=self.arcthicture,
                num_epochs=num_epochs, 
                loss_fn=criterion, 
                optimizer_name=optimizer,
                batch_size=batch_size, 
                learning_rate=learning_rate)
        
        
       

    def fit(self,X_train,X_val,y_train,y_val,
                num_epochs, loss_fn, optimizer_name, batch_size, learning_rate):
        """
        Train the model with the given parameters.
        
        Parameters:
        - num_epochs: Number of epochs to train the model.
        - loss_fn: Loss function to use for training.
        - optimizer: Optimizer to use for training.
        - batch_size: Size of each batch during training.
        - x_valid: Validation input data.
        - y_valid: Validation target data.
        - learning_rate: Learning rate for the optimizer.
        
        Returns:
        - history: Training history containing loss and accuracy metrics.
        """

        # Define datasets for data loaders
        train_ds = TensorDataset(X_train, y_train)
        val_ds = TensorDataset(X_val, y_val)

        train_size = len(train_ds)
        if optimizer_name == 'GD':
            batch_size = train_size
            train_dl = DataLoader(train_ds, batch_size, shuffle=True)
            #val_dl = DataLoader(val_ds, batch_size, shuffle=True)
        
        elif optimizer_name == 'SGD':
            batch_size = 1
            train_dl = DataLoader(train_ds, batch_size, shuffle=True)
            #val_dl = DataLoader(val_ds, batch_size, shuffle=True)

        else:
            batch_size = batch_size
            train_dl = DataLoader(train_ds, batch_size, shuffle=True)
            #val_dl = DataLoader(val_ds, batch_size, shuffle=True)
        
        optimizer_choiche = {
        'GD': torch.optim.SGD(self.arcthicture.parameters(), lr=learning_rate),
        'SGD': torch.optim.SGD(self.arcthicture.parameters(), lr=learning_rate),
        'MiniSGD': torch.optim.SGD(self.arcthicture.parameters(), lr=learning_rate),
        'ASGD': torch.optim.ASGD(self.arcthicture.parameters(), lr=learning_rate),
        
        'RMSprop': torch.optim.RMSprop(self.arcthicture.parameters(), lr=learning_rate),
        'Adam': torch.optim.Adam(self.arcthicture.parameters(), lr=learning_rate)
        }
        optimizer_instance = optimizer_choiche[optimizer_name]
        
        return self.arcthicture.train(
            num_epochs=num_epochs, 
            loss_fn=loss_fn, 
            optimizer=optimizer_instance, 
            train_dl=train_dl, 
            train_size=train_size, 
            batch_size=batch_size,
            x_valid=val_ds[0],
            y_valid=val_ds[1])
        



100%|██████████| 10/10 [00:00<00:00, 96.15it/s]

********************
forward pass layer 1
forward pass layer 2
forward pass layer 3
forward pass layer 4
forward pass layer 5
********************
forward pass layer 1
forward pass layer 2
forward pass layer 3
forward pass layer 4
forward pass layer 5
********************
forward pass layer 1
forward pass layer 2
forward pass layer 3
forward pass layer 4
forward pass layer 5
********************
forward pass layer 1
forward pass layer 2
forward pass layer 3
forward pass layer 4
forward pass layer 5
********************
forward pass layer 1
forward pass layer 2
forward pass layer 3
forward pass layer 4
forward pass layer 5
********************
forward pass layer 1
forward pass layer 2
forward pass layer 3
forward pass layer 4
forward pass layer 5
********************
forward pass layer 1
forward pass layer 2
forward pass layer 3
forward pass layer 4
forward pass layer 5
********************
forward pass layer 1
forward pass layer 2
forward pass layer 3
forward pass layer 4
forward pass 